In [1]:
import pandas as pd
import numpy as np
import scipy.stats
import scipy
from scipy.stats import ttest_ind



In [2]:
def filter_init(data):
  frametrack = 'player::draw()'
  frametrack_data = data[data[:, 0] == frametrack, :]
  last_init = np.argmax(frametrack_data[:,4])
  init_end_time = frametrack_data[last_init, 3]
  return data[data[:, 2] > init_end_time, :]

In [3]:
def load_di(path):
  data = pd.read_csv(path).to_numpy() 
  data = filter_init(data)
  starts = data[:, 2]
  data = data[starts.argsort()]
  name2durations = {}
  for row in data:
    name = row[0]
    duration = row[4]
    if name not in name2durations:
      name2durations[name] = [duration]
    else: 
      name2durations[name] += [duration]
  return name2durations

In [4]:
 di1 = load_di('~/ggpcum/4k1.csv')

In [5]:
 di2 = load_di('~/ggpcum/4k2.csv')

In [6]:
 difhd = load_di('~/ggpcum/fhd1.csv')

In [7]:
def test(d1, d2, sigma0):
  n1 = d1.shape[0]
  n2 = d2.shape[0]
  if (min(n1, n2) < 50):
    return 1.0;
  
  v1 = d1.var() / n1
  v2 = d2.var() / n2
  stat = (d1.mean() - d2.mean()) / np.sqrt(v1 + v2 + sigma0*sigma0)
  p = scipy.stats.norm().cdf(stat)
  if (np.isnan(p)): 
    return 1.0
  return min(p, 1-p) 


In [8]:

def get_pvalues(di1, di2, sigma0):
  pvalues = []
  for name in di1.keys():
    durations1 = np.array(di1[name])
    durations2 = np.array(di2[name])
    pvalue = test(durations1, durations2, sigma0)
    pvalues += [pvalue]
  return np.array(pvalues), np.array(list(di1.keys()))
  

In [12]:
pvalues, name = get_pvalues(di1, difhd, 300)

In [13]:
name[pvalues < 0.05], pvalues[pvalues < 0.05]

(array(['player::draw()',
        'player::vulkan_specific::acquire_next_framebuffer()'],
       dtype='<U51'),
 array([0., 0.]))